<a href="https://colab.research.google.com/github/alzaavi/lda2/blob/main/port2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [9]:
from google.colab import drive
import numpy as np
import cv2
from IPython.display import clear_output

# Clone the GitHub repository containing the video files
!git clone https://github.com/alzaavi/lda2.git

def detect_circle(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    circle_centers = []
    for cnt in contours:
        (a, b), radius = cv2.minEnclosingCircle(cnt)
        if radius > 10:  # Set a minimum radius to filter out small contours
            circle_centers.append((int(a), int(b)))
    
    return circle_centers

def find_first_peak(heights):
    for i in range(1, len(heights) - 1):
        if heights[i-1] < heights[i] > heights[i+1]:
            return i
    return -1

def calculate_coefficient(video_path):
    video_capture = cv2.VideoCapture(video_path)

    ref_point = (0, 480)
    
    success, frame = video_capture.read()
    if not success:
        print("Error reading frame")
        exit()
    
    # Resize the frame
    frame = cv2.resize(frame, (int(frame.shape[1] * 0.5), int(frame.shape[0] * 0.5)))
    
    circle_centers_list = detect_circle(frame)
    height_list = [ref_point[1] - circle_centers_list[0][1]]
    
    frame_idx = 0
    skip_frames_count = 5  # Skip every 5 frames
    
    while video_capture.isOpened():
        success, frame = video_capture.read()
        if not success:
            break

        frame_idx += 1

        # Skip frames
        if frame_idx % skip_frames_count == 0:
            # Resize the frame
            frame = cv2.resize(frame, (int(frame.shape[1] * 0.5), int(frame.shape[0] * 0.5)))

            circle_centers = detect_circle(frame)
            circle_centers_list.extend(circle_centers)
            dist = ref_point[1] - circle_centers_list[-1][1]
            height_list.append(dist)
    
    video_capture.release()

    initial_height = height_list[0]
    first_peak_idx = find_first_peak(height_list)
    bounce_height = min(height_list[first_peak_idx+1:])
    restitution_coeff = np.sqrt(bounce_height / initial_height)
    return restitution_coeff

video_file_names = [
    "IMG_0663 2.mov",
    "IMG_0664 2.mov",
    "IMG_0665 2.mov",
    "IMG_0666 2.mov",
    "IMG_0668 2.mov",
    "IMG_0669 2.mov",
    "IMG_0673 2.mov",
    "IMG_0674 2.mov",
    "IMG_0675 2.mov"
]

for vid_file in video_file_names:
    video_path = f"lda2/{vid_file}"
    coeff = calculate_coefficient(video_path)
    print(f"Coefficient of restitution for {vid_file}: {coeff}")


fatal: destination path 'lda2' already exists and is not an empty directory.
Coefficient of restitution for IMG_0663 2.mov: 0.5944694961101522
Coefficient of restitution for IMG_0664 2.mov: 0.5934988596101457
Coefficient of restitution for IMG_0665 2.mov: 0.5962326324467672
Coefficient of restitution for IMG_0666 2.mov: 0.6014910328186378
Coefficient of restitution for IMG_0668 2.mov: 0.604392598296949
Coefficient of restitution for IMG_0669 2.mov: 0.6028643115587955
Coefficient of restitution for IMG_0673 2.mov: 0.5950577886797546
Coefficient of restitution for IMG_0674 2.mov: 0.5946776072651745
Coefficient of restitution for IMG_0675 2.mov: 0.5885542295962601
